# Project: Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. 

The Factbook contains demographic information like the following:
population — the global population
population_growth — the annual population growth rate, as a percentage.
area — the total land and water area.

In this guided project, we'll use SQL in Jupyter Notebook to analyse data from this database.

I used the factbook.db database.



In [44]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db
    

'Connected: None@factbook.db'

# Query the database

In [47]:

%%sql
SELECT * 
FROM sqlite_master 
WHERE type = 'table';


Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


# Query the facts table
query that returns the first five rows of the facts table in the database..

In [50]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Summary Statistics
Let's start by calculating some summary statistics and look for any outlier countries.


In [54]:
%%sql
SELECT name 
FROM facts
WHERE population = (SELECT MIN (population) FROM facts);

Done.


name
Antarctica


In [55]:
%%sql
SELECT name 
FROM facts
WHERE population = (SELECT MAX(population) FROM facts)

Done.


name
World


# Exploring Outliers
We do not expect the following:

A country with a population of 0
A country with a population of 7,256,490,011 (or more than 7.2 billion people)


The CIA Factbook page for Antarctica says *"no indigenous inhibitants, but there are both permanent and summary-only staffed research stations..."*
The population of the world is over 7.2 billion people.

Let's use subqueries to zoom in on just these countries

Let's recalculate the summary statistics but this time excluding the row for the whole world.



In [56]:
%%sql
SELECT min(population), max(population), min(population_growth), max(population_growth)
from facts
where name != 'World';

Done.


min(population),max(population),min(population_growth),max(population_growth)
0,1367485388,0.0,4.02


In [58]:
%%sql
SELECT avg(population), avg(area)
FROM facts
WHERE name !='World';

Done.


avg(population),avg(area)
32242666.56846473,555093.546184739


# Finding Densely Populated Countries
Let's identify countries that have the following:

Above-average values for population.
Below-average values for area.

In [81]:

%%sql
SELECT name
FROM facts
WHERE population > (SELECT AVG(population)FROM facts 
AND area < (SELECT AVG(area)) from facts;













(sqlite3.OperationalError) near "AND": syntax error
[SQL: SELECT name
FROM facts
WHERE population > (SELECT AVG(population)FROM facts 
AND area < (SELECT AVG(area)) from facts;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [89]:
%%sql
SELECT name
FROM facts
WHERE population > (SELECT AVG(population) FROM facts WHERE name != 'World')
AND area< (SELECT AVG(area) FROM facts WHERE name !='World')


Done.


name
Bangladesh
Germany
Iraq
Italy
Japan
"Korea, South"
Morocco
Philippines
Poland
Spain
